In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
from importHelpers.response import *
from mlxtend.preprocessing import minmax_scaling
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel(r"data\\191126P2PhysData.xlsx", header=None).T
df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.981047,-0.977236,-1.026854,-0.983121,-0.979734,-0.994280,-0.995091,-0.992572,-1.017546,-1.019958,...,-1.093013,-1.070537,-1.077735,-1.060189,-1.096157,-1.136632,-1.083426,-1.066468,-1.031957,-1.045915
1,-0.282811,0.832223,1.571895,1.511685,1.777528,2.218740,2.128655,1.977111,2.087082,2.107779,...,2.047339,2.123366,1.872845,2.199981,2.028838,2.053396,2.005515,2.070698,1.913456,1.785120
2,0.571081,0.494429,0.751718,0.580546,0.632446,0.855266,0.680370,0.764025,0.453674,0.620698,...,0.516698,0.785411,0.556864,0.525525,0.621531,0.685788,0.578357,0.565160,0.614412,0.469929
3,2.096727,3.333588,3.859820,4.164157,4.005630,4.153227,4.078456,3.821792,3.855679,3.537391,...,4.618674,4.476266,4.738743,4.588993,4.546078,4.538284,4.452612,4.481684,4.693524,4.886218
4,0.281441,0.346466,0.693288,0.877922,0.841007,0.955635,1.120212,1.092848,1.156230,1.256585,...,1.031648,0.785741,0.998500,0.974495,0.936173,0.851405,0.943852,0.935190,0.879609,0.934099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.528998,-0.300300,0.127119,-0.089443,-0.031135,-0.247338,-0.502079,-0.564808,-0.207344,-0.461992,...,-0.836536,-0.686312,-0.195539,-0.764492,-0.778708,-0.933984,-0.947261,-0.431995,-0.667395,-0.722359
997,0.569107,0.442400,0.909735,0.588900,0.565182,0.385463,0.725589,0.170486,0.619737,0.366151,...,0.174376,0.571913,0.842858,0.795257,0.080265,-0.009735,0.534825,0.710001,0.818623,0.445271
998,1.053845,1.161893,1.435134,0.944921,1.207642,1.222189,0.812839,0.925631,0.959094,1.367547,...,0.830300,1.131845,0.756282,0.380590,0.560868,0.571769,0.782311,0.779526,0.894593,0.566538
999,-2.228457,-2.156521,-1.765428,-1.958050,-1.932768,-2.309895,-2.375064,-2.559290,-2.220154,-1.976888,...,-2.288136,-2.737645,-2.435367,-2.752299,-2.631126,-2.736884,-2.537682,-2.329767,-2.633113,-2.504849


In [3]:
def transform(initial):
    # remove and subtract baseline
    # c = frameToSecDF(initial.sub(initial['baseline'], axis = 'rows').drop('baseline', axis = 1).T)
    # drop 70
    c = initial
    a = [a - b > 70 for a, b in zip(list(c.max(axis = 1)), list(c.min(axis= 0)))]
    dropped = []
    for i in range(len(a)):
        if not a[i]:
            dropped.append(list(c.T)[i])
    c = c.drop(dropped, axis = 0)
    # -1 1 scale
    last = c[c.columns[-15:]]
    last = last.mean(axis=1)
    ne = c.sub(last, axis = 0)
    n_one = ne.div(ne.abs().max(axis = 1), axis = 0)
    return n_one

In [4]:
transform(df)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
200,-0.088407,0.021549,0.082557,0.153704,0.146183,0.213467,0.191038,0.232529,0.210619,0.208777,...,-0.023976,-0.015688,-0.008690,0.000986,-0.019340,0.009841,0.038544,0.010392,0.006883,-0.014205
201,-0.689408,-0.226015,-0.283872,-0.300016,-0.670334,-0.751574,-0.418707,-0.372843,-0.819364,-0.501801,...,0.188440,0.101940,0.117513,-0.225271,-0.228233,-0.047756,0.013919,-0.034971,0.068877,0.162696
202,-0.990354,-0.923652,-0.974084,-0.960537,-0.967609,-0.978458,-0.972676,-0.944289,-0.980605,-0.985557,...,-0.061029,-0.043281,0.047411,0.072690,0.022534,0.083082,0.069892,-0.018889,-0.048154,-0.011151
203,-0.118293,-0.018627,0.088724,0.143304,0.072810,0.188135,0.247907,0.316354,0.412795,0.466470,...,0.025629,0.046826,0.028405,-0.024202,-0.024152,-0.002955,-0.020558,-0.014055,-0.047772,-0.013971
204,-0.976320,-0.830323,-0.981859,-0.899315,-0.993270,-0.959339,-0.849284,-0.927730,-0.975614,-0.967458,...,0.039170,0.156835,-0.131418,-0.030452,-0.065468,-0.006317,-0.072729,-0.126144,-0.033616,0.022166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.227571,0.491924,0.985981,0.735655,0.803054,0.553143,0.258686,0.186178,0.599372,0.305024,...,-0.127914,0.045731,0.613018,-0.044638,-0.061071,-0.240555,-0.255901,0.339698,0.067597,0.004064
997,0.059328,-0.070069,0.407190,0.079541,0.055319,-0.128216,0.219133,-0.347758,0.111033,-0.147938,...,-0.343786,0.062194,0.338893,0.290281,-0.439896,-0.531806,0.024318,0.203215,0.314144,-0.067138
998,0.219692,0.302821,0.513046,0.135890,0.338020,0.349212,0.034269,0.121048,0.146794,0.461046,...,0.047703,0.279704,-0.009244,-0.298291,-0.159590,-0.151203,0.010782,0.008639,0.097168,-0.155228
999,0.442226,0.528882,1.000000,0.767964,0.798420,0.344125,0.265621,0.043699,0.452228,0.745271,...,0.370336,-0.171151,0.192978,-0.188804,-0.042836,-0.170235,0.069729,0.320187,-0.045230,0.109279


In [5]:
n = df
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(n)
principalDf = pd.DataFrame(data = principalComponents)
pca_n = pd.DataFrame(data = pca.inverse_transform(principalComponents))
pca_n = pca_n.rename(index={a:b for a,b in zip(range(len(list(n.T))),list(n.T))}, columns={a:b for a,b in zip(range(len(list(n))),list(n))})
# comment next line for no PCA
n = pca_n

In [6]:
db = DBSCAN(eps=3, min_samples=2).fit(n)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
dlabels = db.labels_
print("DBSCAN with your params found:")
print(str(max(dlabels + 1)) + " classified labels")
print(str(list(dlabels).count(-1)) + ' unclassified points out of ' + str(len(dlabels)))

DBSCAN with your params found:
23 classified labels
144 unclassified points out of 1001


In [10]:
dlabels

array([ 0, -1,  0, ...,  0, -1,  0], dtype=int32)